Install packages

Add ifind

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64/'
os.environ['JVM_PATH'] = os.environ['JAVA_HOME'] + 'lib/server/libjvm.so'

In [3]:
!export PYTHONPATH="${PYTHONPATH}:./ifind"b

Prepare topics

Run simulations

In [10]:
!rm /workspace/sims/output/*

In [ ]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_nyt_bm25.xml

In [ ]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_bm25.xml

In [6]:
!cd workspace/simiir/ && python run_simiir.py ../sims/simulation_wapo_monoT5.xml

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
SIMULATION 'wapo_monoT5-321-GPT+_perfect_10rpp'
  Simulation Configuration:
    Topic: 321
    Search Interface: PyterrierWebSearchInterface
        index_dir: 
        corpus: wapo
        neural: _monot5

  User Configuration (GPT+_perfect_10rpp):
    Query Generator: PredeterminedQueryGenerator
        stopword_file: /workspace/data/stopword-list.txt
        query_file: /workspace/data/wapo/gpt+.txt
        user: 1
    Snippet Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.0
        data_handler: 0
    Document Classifier: StochasticInformedTrecTextClassifier
        qrel_file: /workspace/data/wapo/wapo_qrels
        rprob: 1.0
        nprob: 0.0
        data_handler: 0
    Stopping Decision Maker: FixedDep

# Evaluations

## Modalities

### sDCG 

## Browsing behavior

### sDCG